# HW 2 - Разложение матриц градиентным методом

Цель задания: В ходе реализации [разложения Таккера](https://proceedings.neurips.cc/paper/2018/file/45a766fa266ea2ebeb6680fa139d2a3d-Paper.pdf) градиентным методом освоить pyTorch и реализовать подходы оптимизации параметров модели (в отсутствии готовых решений).

[Более-менее внятное описание алгоритма канонического разложения](https://www.alexejgossmann.com/tensor_decomposition_tucker/) - само аналитическое разложение вам реализовывать НЕ НУЖНО

In [216]:
import random
import time
import torch
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from numpy.linalg import svd, matrix_rank, pinv, inv
from scipy.linalg import eigh, eig
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from torch import nn
import tensorly
from torch.nn.functional import mse_loss
from torch.optim import Adam

torch.manual_seed(0)

## 1 Создайте 3х мерный тензор
Размер тензора не меньше 100 по каждой из размерностей.

Заполните случайными целыми числами в диапазоне от 0 до 9.

Примечание: разложение будет корректно работать со случайным тензором, только если изначально создавать случайные ядро и матрицы, а потом по ним формировать тензор. Работайте с типом *torch.Tensor.double*.

In [217]:
# Создадим тензор: размер тензора и r задаётся
def get_tensor(size=(100,200,150), r=10):
    # data - тензор с заданной размерностью
    # U - список матриц
    # G - ядро разложения
    
    U = [np.random.randint(0,10,(n, r)) for n in size]
    G = np.random.randint(0,10,(r, r, r))
    data = torch.tensor(tensorly.tenalg.multi_mode_dot(G, U))
    
    return data, U, G

Сгенерируйте тензор и добавьте к нему случайный шум с размерностью *1e-2*

In [218]:
data, U, G = get_tensor()
data.shape, [u.shape for u in U], G.shape

(torch.Size([100, 200, 150]), [(100, 10), (200, 10), (150, 10)], (10, 10, 10))

In [219]:
data_noise = data.add(torch.rand(data.shape) / 10)

Вопрос:
Почему задание не имеет смысла для полностью случайного тензора и зачем добавлять шум? *не отвечать нельзя*

Ответ:

## 2 Реализуйте метод для восстановления тензора по разложению

In [220]:
# Функция, восстанавливающая тензор по ядру и матрицам
def repair_tensor(G_, U):    
    data = G_.clone()
    for i in range(len(U)):  
        modes = [U[i].shape[0]] + [list(data.shape)[s] for s in range(len(list(data.shape))) if s != i]
        data_i = torch.reshape(torch.moveaxis(data, i, 0), (data.shape[i], -1))
        data = torch.moveaxis(torch.reshape(U[i] @ data_i, modes), 0, i)
    return data

In [221]:
size, r = (100,200,150), 10
U = [torch.randint(0,10,(n, r)).float() for n in size]
G = torch.randint(0,10,(r, r, r)).float()

In [222]:
my_data = repair_tensor(G, U)
my_data_noise = my_data.add(torch.rand(my_data.shape) / 10)

In [223]:
my_data.max()

tensor(1383021.)

In [224]:
def create_tensor(size=(100,200,150), r=10):
    U = [torch.randint(0,10,(n, r)).float() for n in size]
    G = torch.randint(0,10,(r, r, r)).float()
    data = repair_tensor(G, U)
    
    return data, U, G

## 3 Сделайте разложение библиотечным методом
Пакет можете брать любой

In [225]:
G_lib, U_lib = tensorly.decomposition.tucker(tensorly.tensor(my_data_noise), G.shape)

In [226]:
data_lib = torch.tensor(tensorly.tenalg.multi_mode_dot(G_lib, U_lib))

Не забудьте померить ошибку разложения по метрике MSE

In [227]:
mse_loss(my_data_noise, data_lib)

tensor(0.0266)

## 4 Реализуйте разложение градиентным методом

### 4.1 Реализуйте *optimizer*
Можно взять из исходников *PyTorch* и отнаследоваться от *torch.optim.optimizer*.
Используйте квадратичный *Loss*.

In [229]:
class AdamOpt(Optimizer):
    def __init__(self, params, lr=0.001, betas=(0.9, 0.999), eps=1e-1):
        defaults = dict(lr=lr, betas=betas, eps=eps)
        super(AdamOpt, self).__init__(params, defaults)

    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                grad = p.grad.data
                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['m'] = torch.zeros_like(p)
                    state['v'] = torch.zeros_like(p)

                beta1, beta2 = group['betas']
                state['step'] += 1

                m = beta1 * state['m'] + (1 - beta1) * grad
                v = beta2 * state['v'] + (1 - beta2) * grad**2
                m_hat = m / (1 - beta1**state['step'])
                v_hat = v / (1 - beta2**state['step'])

                p.data -= group['lr'] * m_hat / (torch.sqrt(v_hat) + group['eps'])

                state['m'], state['v'] = m, v

### 4.2 Реализуйте цикл оптимизации параметров

Стоит параметры оптимизировать сразу на GPU

In [230]:
def optimization(G, U, data, opt, it, err):
    optim = opt
    f = True
    for i in range(it):    
        optim.zero_grad()
        loss = mse_loss(repair_tensor(G, U), data)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(opt.param_groups[0]['params'], 10**5)
        optim.step()

        if i % 100 == 0:
            print(f'itr {i} mse is {loss}')

        if loss.data < err:
            break 

## 5 Приведите сравнение скорости работы и ошибки восстановления методом из пакета и реализованного градиентного
Сравнение может считаться ± объективным с размером выборки от 10.

In [231]:
size = (100, 120, 110)
r = 10
itr = 50000
eps = 0.5

data, U, G = create_tensor(size, r)
noise = torch.rand(data.shape) / 10
data = noise.add(data)

G_opt = torch.randint(0, 10, (r, r, r)).float().requires_grad_()
U_opt = [torch.randint(0, 10, (s, r)).float().requires_grad_() for s in size]

my_optimizer = AdamOpt([G_opt, *U_opt])

In [232]:
start_time = time.time()
optimization(G_opt, U_opt, data, my_optimizer, itr, eps)
whole_time = time.time()-start_time

itr 0 mse is 47669108736.0
itr 100 mse is 41341739008.0
itr 200 mse is 36216958976.0
itr 300 mse is 31865890816.0
itr 400 mse is 28056180736.0
itr 500 mse is 24660692992.0
itr 600 mse is 21605920768.0
itr 700 mse is 18845620224.0
itr 800 mse is 16348451840.0
itr 900 mse is 14091745280.0
itr 1000 mse is 12058083328.0
itr 1100 mse is 10233184256.0
itr 1200 mse is 8604620800.0
itr 1300 mse is 7161026048.0
itr 1400 mse is 5891616768.0
itr 1500 mse is 4785875968.0
itr 1600 mse is 3833253376.0
itr 1700 mse is 3022899712.0
itr 1800 mse is 2343432704.0
itr 1900 mse is 1782855168.0
itr 2000 mse is 1328659328.0
itr 2100 mse is 968092736.0
itr 2200 mse is 688443008.0
itr 2300 mse is 477242752.0
itr 2400 mse is 322492192.0
itr 2500 mse is 213046848.0
itr 2600 mse is 139047424.0
itr 2700 mse is 91977704.0
itr 2800 mse is 64236076.0
itr 2900 mse is 49047632.0
itr 3000 mse is 40817656.0
itr 3100 mse is 35797516.0
itr 3200 mse is 34303984.0
itr 3300 mse is 33559428.0
itr 3400 mse is 32985942.0
itr 350

In [233]:
%%time
tG, tU = tensorly.decomposition.tucker(tensorly.tensor(data), G.shape)
tdata = torch.tensor(tensorly.tenalg.multi_mode_dot(tG, tU))
print(f'mse for lib optimizer: {mse_loss(tdata, data)}')

mse for lib optimizer: 0.025142613798379898
Wall time: 761 ms


In [234]:
data_opt = torch.tensor(repair_tensor(G_opt, U_opt))
print(f'mse for my optimizer: {mse_loss(data_opt, data)}')

mse for my optimizer: 77.49583435058594


C:\Users\0717~1\AppData\Local\Temp/ipykernel_13616/3178578322.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_opt = torch.tensor(repair_tensor(G_opt, U_opt))
